## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis =1,inplace = True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_df = data.iloc[0:1000]

In [11]:
data_df.shape

(1000, 5)

In [12]:
data_df['volume'] = data_df['volume'] / 1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
X = data_df.drop(['volume'], axis = 1)
y = data_df['volume']

In [15]:
X.shape, y.shape

((1000, 4), (1000,))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state = 5)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train =np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_train =np.array(y_train).astype('float32')
y_test = np.array(y_test).astype('float32')

In [0]:
y_train = y_train.reshape(-1, 1)

In [20]:
y_train.shape

(700, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,1))
b1 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction (x, w, b):
  xw_matmul = tf.matmul(x, w)
  net = tf.add(xw_matmul, b)
  predict = tf.sigmoid(net)
  return predict

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(X_train, y_train, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as tape:
      tape.watch([w, b])
      
      current_prediction = prediction(X_train, w, b)
      current_loss = loss(current_prediction, y_train)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = tape.gradient(current_loss, [w, b])
    
    #Update Weights at output layer
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b, current_loss

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [26]:
for ii in range(100):
    w1, b1, current_loss = train(X_train, y_train, w1, b1)
    print("Training loss at epoch {:d}: {:.3f}".format(ii, current_loss))

Training loss at epoch 0: 252.179
Training loss at epoch 1: 252.076
Training loss at epoch 2: 251.975
Training loss at epoch 3: 251.875
Training loss at epoch 4: 251.777
Training loss at epoch 5: 251.681
Training loss at epoch 6: 251.588
Training loss at epoch 7: 251.496
Training loss at epoch 8: 251.407
Training loss at epoch 9: 251.319
Training loss at epoch 10: 251.235
Training loss at epoch 11: 251.152
Training loss at epoch 12: 251.072
Training loss at epoch 13: 250.995
Training loss at epoch 14: 250.919
Training loss at epoch 15: 250.846
Training loss at epoch 16: 250.776
Training loss at epoch 17: 250.708
Training loss at epoch 18: 250.642
Training loss at epoch 19: 250.578
Training loss at epoch 20: 250.517
Training loss at epoch 21: 250.457
Training loss at epoch 22: 250.400
Training loss at epoch 23: 250.345
Training loss at epoch 24: 250.292
Training loss at epoch 25: 250.240
Training loss at epoch 26: 250.190
Training loss at epoch 27: 250.143
Training loss at epoch 28: 250

In [27]:
for ii in range(0, 100, 5):
    w1, b1, current_loss = train(X_train, y_train, w1, b1)
    print("Training loss at epoch {:d}: {:.3f}".format(ii, current_loss))

Training loss at epoch 0: 248.815
Training loss at epoch 5: 248.808
Training loss at epoch 10: 248.802
Training loss at epoch 15: 248.796
Training loss at epoch 20: 248.791
Training loss at epoch 25: 248.785
Training loss at epoch 30: 248.779
Training loss at epoch 35: 248.774
Training loss at epoch 40: 248.768
Training loss at epoch 45: 248.763
Training loss at epoch 50: 248.758
Training loss at epoch 55: 248.753
Training loss at epoch 60: 248.748
Training loss at epoch 65: 248.743
Training loss at epoch 70: 248.738
Training loss at epoch 75: 248.733
Training loss at epoch 80: 248.729
Training loss at epoch 85: 248.724
Training loss at epoch 90: 248.720
Training loss at epoch 95: 248.715


### Get the shapes and values of W and b

In [28]:
w1, b1

(<tf.Tensor: id=4206, shape=(4, 1), dtype=float32, numpy=
 array([[0.95462024],
        [1.13171   ],
        [0.20968984],
        [0.6468802 ]], dtype=float32)>,
 <tf.Tensor: id=4209, shape=(1,), dtype=float32, numpy=array([1.2941393], dtype=float32)>)

### Model Prediction on 1st Examples in Test Dataset

In [29]:
y_pred = prediction(X_test, w1, b1)
y_pred * 1000000

<tf.Tensor: id=4215, shape=(300, 1), dtype=float32, numpy=
array([[940879.44],
       [941115.1 ],
       [940803.5 ],
       [940981.5 ],
       [940780.  ],
       [940805.  ],
       [941011.1 ],
       [941189.5 ],
       [940917.75],
       [940856.56],
       [940820.7 ],
       [940841.8 ],
       [940840.1 ],
       [941008.3 ],
       [940996.8 ],
       [940915.1 ],
       [941036.1 ],
       [940798.9 ],
       [941273.2 ],
       [940794.1 ],
       [941019.6 ],
       [940827.06],
       [940910.7 ],
       [941040.9 ],
       [940912.94],
       [940897.94],
       [941097.6 ],
       [941046.4 ],
       [940900.8 ],
       [940959.6 ],
       [941065.9 ],
       [940904.1 ],
       [940834.44],
       [940820.1 ],
       [940907.6 ],
       [940878.1 ],
       [940948.94],
       [941012.2 ],
       [940949.7 ],
       [940987.44],
       [940895.2 ],
       [940970.9 ],
       [940899.4 ],
       [940880.6 ],
       [940969.7 ],
       [940817.  ],
       [940904.75],
 

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [31]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
dummy = pd.get_dummies(iris['Species'])

In [0]:
iris = pd.concat([iris, dummy], axis=1)

In [0]:
iris.drop(labels=['Id', 'Species'], axis=1, inplace=True)

In [35]:
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = iris.drop(labels=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], axis=1)
y = iris.drop(labels=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], axis=1)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()

In [0]:
X = scaler.fit_transform(X)

In [0]:
y = y.values

In [41]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state = 99)

In [0]:
# define base model
model = tf.keras.models.Sequential()
# Normalize the data
model.add(tf.keras.layers.BatchNormalization())
# Add the hidden layer
model.add(tf.keras.layers.Dense(3, input_dim=4, activation='softmax'))
# Compile model
model.compile( optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [44]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, batch_size = X_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100


W1203 09:28:38.293096 140665610463104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:12: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



105/105 [==============================] - 1s 11ms/sample - loss: 0.6898 - acc: 0.6095 - val_loss: 0.7350 - val_acc: 0.6222
Epoch 2/100
105/105 [==============================] - 0s 137us/sample - loss: 0.6860 - acc: 0.6095 - val_loss: 0.7313 - val_acc: 0.6222
Epoch 3/100
105/105 [==============================] - 0s 125us/sample - loss: 0.6822 - acc: 0.6095 - val_loss: 0.7277 - val_acc: 0.6222
Epoch 4/100
105/105 [==============================] - 0s 116us/sample - loss: 0.6785 - acc: 0.6095 - val_loss: 0.7241 - val_acc: 0.6222
Epoch 5/100
105/105 [==============================] - 0s 128us/sample - loss: 0.6748 - acc: 0.6095 - val_loss: 0.7206 - val_acc: 0.6222
Epoch 6/100
105/105 [==============================] - 0s 122us/sample - loss: 0.6712 - acc: 0.6095 - val_loss: 0.7172 - val_acc: 0.6444
Epoch 7/100
105/105 [==============================] - 0s 100us/sample - loss: 0.6677 - acc: 0.6095 - val_loss: 0.7137 - val_acc: 0.6444
Epoch 8/100
105/105 [==============================] -

### Model Prediction

In [0]:
y_pred = model.predict(X_test)

In [46]:
y_pred

array([[0.06689516, 0.33502224, 0.59808254],
       [0.71006113, 0.19277047, 0.09716837],
       [0.19120799, 0.38621882, 0.4225732 ],
       [0.05070156, 0.3112746 , 0.63802385],
       [0.9678472 , 0.01818296, 0.01396976],
       [0.04424505, 0.5326995 , 0.4230554 ],
       [0.0955571 , 0.42313498, 0.48130792],
       [0.01325559, 0.44650623, 0.54023814],
       [0.90581894, 0.0551342 , 0.03904688],
       [0.0984573 , 0.36364603, 0.5378967 ],
       [0.02374434, 0.35033193, 0.6259237 ],
       [0.03583222, 0.535475  , 0.42869282],
       [0.06398857, 0.3947252 , 0.54128623],
       [0.06996042, 0.488322  , 0.44171757],
       [0.07560674, 0.30904615, 0.61534715],
       [0.98738617, 0.00635763, 0.00625621],
       [0.87697905, 0.07814807, 0.04487296],
       [0.15106714, 0.42792347, 0.4210094 ],
       [0.06542039, 0.5593542 , 0.37522545],
       [0.01003288, 0.5258347 , 0.46413243],
       [0.07818997, 0.43283594, 0.48897403],
       [0.25251517, 0.31382632, 0.43365857],
       [0.

### Getting the index of maximum probability for every test row


In [47]:
y_pred_max = np.argmax(y_pred, axis=1)
y_pred_max

array([2, 0, 2, 2, 0, 1, 2, 2, 0, 2, 2, 1, 2, 1, 2, 0, 0, 1, 1, 1, 2, 2,
       0, 1, 2, 0, 0, 2, 2, 2, 0, 2, 0, 1, 1, 0, 0, 2, 2, 2, 2, 1, 2, 2,
       1])

### Creating a blank array for setting the predicted classes

In [0]:
y_pred_class = np.zeros(shape=y_test.shape, dtype='int32')

### Setting the Class as True for those index with maximum probability

In [0]:
for ii in range(y_pred.shape[0]):
  y_pred_class[ii, y_pred_max[ii]] = 1

### These are predicted classes with the model:

In [50]:
y_pred_class

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]], dtype=int32)

### Reset the Class labels for the dummy columns

In [0]:
dummy_cols = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

classes = pd.DataFrame()

for ii in range(y_pred.shape[0]):
  classes.loc[ii, 'Species'] = dummy_cols[y_pred_max[ii]]

### These are the predicted Classes

In [52]:
classes

,Species
0,Iris-virginica
1,Iris-setosa
2,Iris-virginica
3,Iris-virginica
4,Iris-setosa
5,Iris-versicolor
6,Iris-virginica
7,Iris-virginica
8,Iris-setosa
9,Iris-virginica


### Save the Model

In [0]:
model.save('iris_model-santosh.html')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
# define base model
model2 = tf.keras.models.Sequential()
# Normalize the data
model2.add(tf.keras.layers.BatchNormalization())
# Add 2 more hidden layer with relu activation function:
model2.add(tf.keras.layers.Dense(4, input_dim=4, activation='relu'))
model2.add(tf.keras.layers.Dense(3, input_dim=4, activation='relu'))
model2.add(tf.keras.layers.Dense(3, input_dim=3, activation='softmax'))
# Compile model
model2.compile( optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
model2.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, batch_size = X_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 4ms/sample - loss: 1.0198 - acc: 0.3429 - val_loss: 1.0360 - val_acc: 0.3778
Epoch 2/100
105/105 [==============================] - 0s 139us/sample - loss: 1.0168 - acc: 0.3524 - val_loss: 1.0328 - val_acc: 0.3778
Epoch 3/100
105/105 [==============================] - 0s 107us/sample - loss: 1.0139 - acc: 0.3524 - val_loss: 1.0296 - val_acc: 0.3778
Epoch 4/100
105/105 [==============================] - 0s 119us/sample - loss: 1.0109 - acc: 0.3619 - val_loss: 1.0264 - val_acc: 0.3778
Epoch 5/100
105/105 [==============================] - 0s 115us/sample - loss: 1.0080 - acc: 0.3619 - val_loss: 1.0233 - val_acc: 0.3778
Epoch 6/100
105/105 [==============================] - 0s 106us/sample - loss: 1.0050 - acc: 0.3619 - val_loss: 1.0203 - val_acc: 0.3778
Epoch 7/100
105/105 [==============================] - 0s 107us/sample - loss: 1.0022 - acc: 0.3810 - val_loss: 1.0173 - val_acc: 0.377

### As we can see, Testing accuracy with single output layer was 80%. After adding 2 hidden layers, the Testing accuracy went to 82.22%.
### By adding more hidden layers, the classification algorithm will have more decision surfaces and so better chances of classifying the given dataset  